In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

In [3]:
train = pd.read_csv('dataset/hatespeech-train.csv')
test = pd.read_csv('dataset/hatespeech-test.csv')
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
def  cleaning_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

In [5]:
test_clean  = cleaning_text(test, "tweet")
train_clean = cleaning_text(train,"tweet")

In [6]:
from sklearn.utils import resample
train_major = train_clean[train_clean.label==0]
train_minor = train_clean[train_clean.label==1]
train_minor_upsampled = resample(train_minor,replace=True,n_samples=len(train_major),random_state=123)
train_upsampled = pd.concat([train_minor_upsampled, train_major])
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

In [7]:
pipeline_SGD = Pipeline([('vect', CountVectorizer()),('tfidf',  TfidfTransformer()),('nb', SGDClassifier()),])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],train_upsampled['label'],random_state = 0)

In [9]:
model = pipeline_SGD.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [10]:
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9694666666666667